<a href="https://colab.research.google.com/github/NegroAmigo/BachelorThesis/blob/main/bachelors_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics==8.2.103 -q
!pip install roboflow
!pip install -U ultralytics -q
!pip install wandb

In [1]:
import ultralytics
from ultralytics import YOLO
from ultralytics import settings

from roboflow import Roboflow
from google.colab import drive

import os
import yaml
import shutil
import cv2
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import torch
import csv
from difflib import SequenceMatcher
import pandas as pd
import random


rf = Roboflow(api_key="vzRLQrA2LEpOdmEObpb1")
project = rf.workspace("bachelorsworkspace").project("3d-fdm-failures")
version = project.version(7)
dataset = version.download("yolov8")


drive.mount('/content/drive')


settings.update({"wandb": True})


loading Roboflow workspace...
loading Roboflow project...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = YOLO('/content/drive/MyDrive/Default_Train_pr/run_2/weights/best.pt')

results = model('/content/3d-fdm-failures-6/valid/images/4-20-3-_jpg.rf.a03e4668ee3511f4387652e3baf20a3e.jpg', save=True, conf = 0.35)

for r in results:
    print(r.boxes)

# **Drawing Predicted box with IoU**

Scripts to draw boxes on original image with IoU and IoM and prediction class

In [ ]:
def draw_boxes(image, boxes, classes, color=(0, 255, 0), label_map=None, is_predicted=None, confidences=None):
    img_copy = image.copy()
    h, w = img_copy.shape[:2]
    for i, (box, class_id) in enumerate(zip(boxes, classes)):
        x_center, y_center, width, height = box
        print(f"Class: {class_id} x_center: {x_center} y_center: {y_center} width: {width} height: {height} " )
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)

        print(f"Mins: {x_min} {y_min} Max: {x_max} {y_max}")
        cv2.rectangle(img_copy, (x_min, y_min), (x_max, y_max), color, 2)

        if is_predicted:
            confidence = confidences[i] if confidences is not None else 0.0
            label = f"{label_map[int(class_id)]} {confidence:.2f}" if label_map else f"Class {int(class_id)} {confidence:.2f}"
        else:
            label = label_map[int(class_id)] if label_map else f"Class {int(class_id)}"
        (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        if y_min - text_height - baseline < 0:
            label_y_min = y_max + text_height + baseline
            cv2.rectangle(img_copy, (x_min, y_max + baseline ), (x_min + text_width, y_max + text_height + baseline), (0, 0, 0), thickness=cv2.FILLED)
            cv2.putText(img_copy, label, (x_min, y_max + text_height ), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        else:
            cv2.rectangle(img_copy, (x_min, y_min - text_height - baseline - 10), (x_min + text_width, y_min), (0, 0, 0), thickness=cv2.FILLED)
            cv2.putText(img_copy, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return img_copy

In [ ]:
def yolo_to_coords(box, img_width, img_height):
        x_center, y_center, width, height = box
        width_half = (width * img_width) / 2
        height_half = (height * img_height) / 2
        x_min = (x_center * img_width) - width_half
        y_min = (y_center * img_height) - height_half
        x_max = (x_center * img_width) + width_half
        y_max = (y_center * img_height) + height_half
        return np.array([x_min, y_min, x_max, y_max])

def calculate_iou(boxA, boxB, img_width, img_height):
    # convert boxes from yolo to corner coord
    boxA_corners = yolo_to_coords(boxA, img_width, img_height)
    boxB_corners = yolo_to_coords(boxB, img_width, img_height)

    xA = np.maximum(boxA_corners[0], boxB_corners[0])
    yA = np.maximum(boxA_corners[1], boxB_corners[1])
    xB = np.minimum(boxA_corners[2], boxB_corners[2])
    yB = np.minimum(boxA_corners[3], boxB_corners[3])

    # compute intersection of rectangles
    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)

    # compute area of prediction and ground truth boxes
    boxAArea = (boxA_corners[2] - boxA_corners[0]) * (boxA_corners[3] - boxA_corners[1])
    boxBArea = (boxB_corners[2] - boxB_corners[0]) * (boxB_corners[3] - boxB_corners[1])

    # compute IoU
    iou = interArea / (boxAArea + boxBArea - interArea)

    return iou

def calculate_iom(boxA, boxB, img_width, img_height):
    # convert boxes from yolo to corner coord
    boxA_corners = yolo_to_coords(boxA, img_width, img_height)
    boxB_corners = yolo_to_coords(boxB, img_width, img_height)

    xA = np.maximum(boxA_corners[0], boxB_corners[0])
    yA = np.maximum(boxA_corners[1], boxB_corners[1])
    xB = np.minimum(boxA_corners[2], boxB_corners[2])
    yB = np.minimum(boxA_corners[3], boxB_corners[3])

    # compute intersection of rectangles
    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)

    # compute area of prediction and ground truth boxes
    boxAArea = (boxA_corners[2] - boxA_corners[0]) * (boxA_corners[3] - boxA_corners[1])
    boxBArea = (boxB_corners[2] - boxB_corners[0]) * (boxB_corners[3] - boxB_corners[1])

    # compute the intersection over minimum
    minArea = min(boxAArea, boxBArea)
    iom = interArea / minArea

    return iom

def compare_boxes(pred_boxes, gt_boxes, predicted_classes, ground_truth_classes, img_width, img_height, iou_threshold=0.5,iom_threshold=0.5):
  matches = []
  unmatched_pred = []
  unmatched_gt = []

  # create matrix of IoU between all predicted and ground truth boxes
  iou_matrix = np.zeros((len(pred_boxes), len(gt_boxes)))
  iom_matrix = np.zeros((len(pred_boxes), len(gt_boxes)))

  for i, pred_box in enumerate(pred_boxes):
      for j, gt_box in enumerate(gt_boxes):
          iou_matrix[i, j] = calculate_iou(pred_box, gt_box, img_width, img_height)
          iom_matrix[i, j] = calculate_iom(pred_box, gt_box, img_width, img_height)

  # match predictions with ground truth boxes on highest IoU
  pred_matched = np.zeros(len(pred_boxes), dtype=bool)
  gt_matched = np.zeros(len(gt_boxes), dtype=bool)

  # iterate over predictions to find best match in ground truth
  for i in range(len(pred_boxes)):
        best_gt_idx = np.argmax(iou_matrix[i])
        best_iou = iou_matrix[i, best_gt_idx]
        best_iom = iom_matrix[i, best_gt_idx]

        # if both IoU and IoM exceed their thresholds and ground truth is not already matched
        if best_iou >= iou_threshold and best_iom >= iom_threshold and not gt_matched[best_gt_idx]:
            #matches.append((pred_boxes[i], gt_boxes[best_gt_idx], best_iou, best_iom))
            matches.append((pred_boxes[i],gt_boxes[best_gt_idx], best_iou, best_iom, predicted_classes[i], ground_truth_classes[best_gt_idx]))
            pred_matched[i] = True
            gt_matched[best_gt_idx] = True

    # collect unmatched predictions and ground truth boxes
  unmatched_pred = [pred_boxes[i] for i in range(len(pred_boxes)) if not pred_matched[i]]
  unmatched_gt = [gt_boxes[j] for j in range(len(gt_boxes)) if not gt_matched[j]]

  return matches, unmatched_pred, unmatched_gt

In [ ]:
def place_label(img, label, x_min, boundary_y, color, align_above=False):
        h, w = img.shape[:2]
        (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)

        # determine y-coordinate for label above or below boundary
        if align_above:  # try to place label above the top boundary for predicted labels
            label_y_min = boundary_y - baseline  # smaller gap by only subtracting baseline
            if label_y_min < text_height:  # check if label is out of bounds above
                label_y_min = boundary_y + text_height + baseline  # place below if above is out of bounds
        else:  # place label slightly below bottom boundary for ground truth labels
            label_y_min = boundary_y + baseline + 15 # smaller gap by using only baseline offset
            if label_y_min + text_height > h:  # check if label is out of bounds below
                label_y_min = boundary_y - text_height - baseline  # place above if below is out of bounds

        # adjust x-coordinate if label goes beyond image width
        label_x_min = max(0, min(x_min, w - text_width))

        # draw filled rectangle background and text
        cv2.rectangle(img, (label_x_min, label_y_min - text_height - baseline),
                      (label_x_min + text_width, label_y_min), (0, 0, 0), thickness=cv2.FILLED)
        cv2.putText(img, label, (label_x_min, label_y_min - 2), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)



def draw_boxes_with_iou(image_path, predicted_boxes, predicted_classes, ground_truth_boxes, ground_truth_classes, label_map, confidences=None):
    img_copy = cv2.imread(image_path).copy()
    h, w = img_copy.shape[:2]

    # calculate IoU between predicted and ground-truth boxes
    matches, unmatched_pred, unmatched_gt = compare_boxes(predicted_boxes, ground_truth_boxes,predicted_classes, ground_truth_classes, w, h)
    cls_matches = []



    # draw ground truth boxes (green)
    for i, (gt_box, gt_class) in enumerate(zip(ground_truth_boxes, ground_truth_classes)):
        x_center, y_center, width, height = gt_box
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)

        matched_id = None
        for match_idx, (_, match_gt_box, _, _,_,_) in enumerate(matches):
            if np.array_equal(match_gt_box, gt_box):
                matched_id = match_idx
                break

        # matched ID to label if box is matched
        if matched_id is not None:
            label = f"{matched_id}: {label_map[int(gt_class)]}"
        else:
            label = f"{label_map[int(gt_class)]}"

        cv2.rectangle(img_copy, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        #label = f"{i}: {label_map[int(ground_truth_classes[i])]}" if label_map else f"Class {int(ground_truth_classes[i])}"
        # (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        # cv2.rectangle(img_copy, (x_min, y_max + text_height + baseline ), (x_min + text_width, y_max), (0, 0, 0), thickness=cv2.FILLED)
        # cv2.putText(img_copy, label, (x_min, y_max + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        place_label(img_copy, label, x_min, y_max, (0, 255, 0))

    # draw predicted boxes (red) and IoU values
    for i, ((pred_box, gt_box, iou, iom, _, _), conf) in enumerate(zip(matches, confidences)):
        x_center, y_center, width, height = pred_box
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)

        cv2.rectangle(img_copy, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Blue for predicted

        # Find corresponding class ID
        for idx, (p_box, g_box, _, _, pred_cls, gt_cls) in enumerate(matches):
            if np.array_equal(p_box, pred_box) and np.array_equal(g_box, gt_box):
                class_id = predicted_classes[idx]
                cls_matches.append((idx, int(pred_cls), gt_cls, os.path.basename(image_path)))
                #cls_matches.append((idx, int(class_id), ground_truth_classes[idx]))
                break

        label = f"{i}: {label_map[int(class_id)]} {conf:.2f} IoU: {iou:.2f}" #IoM: {iom:.2f}"
        # (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        # cv2.rectangle(img_copy, (x_min, y_max + text_height + baseline), (x_min + text_width, y_max), (0, 0, 0), thickness=cv2.FILLED)
        # cv2.putText(img_copy, label, (x_min, y_max +15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        place_label(img_copy, label, x_min, y_min, (255, 255, 255), True)
    return img_copy, cls_matches




# **STATISTICAL SCRIPTS**

Scripts which create .csv file for further analysis on which images model performs better/worse.

`statistics_on_cls_matches` - creates csv. with models classification predictions

`statistics_on_no_detection` - creates .csv with images where model did not detected any error


In [ ]:
def statistics_on_cls_matches(images_folder, model_path):

  model = YOLO(model_path)

  for image_name in os.listdir(images_folder):
    image_path = os.path.join(images_folder, image_name)
    results = model(cv2.imread(image_path), conf=0.35, augment=False)

    predicted_boxes = results[0].boxes.xywhn
    predicted_classes = results[0].boxes.cls
    predicted_confidence = results[0].boxes.conf


    ground_truth_path = image_path.replace('.jpg', '.txt').replace('images', 'labels')
    if not os.path.exists(ground_truth_path): continue
    ground_truth_boxes = []
    ground_truth_classes = []

    with open(ground_truth_path, 'r') as f:
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.split())
            ground_truth_boxes.append([x_center, y_center, width, height])
            ground_truth_classes.append(int(class_id))

    ground_truth_boxes = np.array(ground_truth_boxes)
    predicted_boxes = predicted_boxes.cpu().numpy()

    label_map = {0: "Blobs", 1: "Cracks", 2: "Spaghetti", 3: "Stringging", 4: "Under Extrusion"}

    cls_matches_list =[]

    predicted_img, cls_matches_list = draw_boxes_with_iou(image_path, predicted_boxes, predicted_classes, ground_truth_boxes, ground_truth_classes, label_map, confidences=results[0].boxes.conf)

    output_csv_path = "cls_matches.csv"

    file_exists = os.path.exists(output_csv_path) and os.path.getsize(output_csv_path) > 0

    with open(output_csv_path, mode='a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        if not file_exists:
              csv_writer.writerow(["Match_ID", "Predicted_Class", "Ground_Truth_Class", "Image_Name"])

        csv_writer.writerows(cls_matches_list)


In [ ]:
statistics_on_cls_matches('/content/3d-fdm-failures-7/valid/images', '/content/drive/MyDrive/BP_Experiments/Baseline_run/weights/best.pt')

In [ ]:
def statistics_on_no_detection(images_folder, model_path):
  model = YOLO(model_path)
  total_instances = 0;
  for image_name in os.listdir(images_folder):
    image_path = os.path.join(images_folder, image_name)
    results = model(cv2.imread(image_path), conf=0.35, augment=False)


    predicted_boxes = results[0].boxes.xywhn
    predicted_classes = results[0].boxes.cls
    predicted_confidence = results[0].boxes.conf
    ground_truth_path = image_path.replace('.jpg', '.txt').replace('images', 'labels')
    if not os.path.exists(ground_truth_path): continue
    ground_truth_boxes = []
    ground_truth_classes = []

    with open(ground_truth_path, 'r') as f:
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.split())
            ground_truth_boxes.append([x_center, y_center, width, height])
            ground_truth_classes.append(int(class_id))

    total_instances += len(ground_truth_boxes)

    label_map = {0: "Blobs", 1: "Cracks", 2: "Spaghetti", 3: "Stringging", 4: "Under Extrusion"}

    if predicted_boxes.shape[0] == 0:

      print(f"No detection for image: {image_name}")
      output_csv_path = "cls_no_detection.csv"

      class_names = [label_map[class_id] for class_id in ground_truth_classes]

      file_exists = os.path.exists(output_csv_path) and os.path.getsize(output_csv_path) > 0

      with open(output_csv_path, mode='a', newline='') as csv_file:
          csv_writer = csv.writer(csv_file)

          if not file_exists:
                csv_writer.writerow(["Ground_Truth_Classes", "Image_Name"])

          for class_name in class_names:
            csv_writer.writerow([class_name, os.path.basename(image_path)])
    print(total_instances)


In [ ]:
statistics_on_no_detection('/content/3d-fdm-failures-6/valid/images', '/content/drive/MyDrive/NoAug_NoMosaic_CosLR_pr/run_2/weights/best.pt')

# **DIAGNOSTIC MATRIX SCRIPT**

Script which creates matrices of models predictions. Each mosaic represents the model predictions on a picture from the validation split of a created dataset. The mosaic is divided into 4 parts, where the top left picture is an example of misclassification, the top right picture is low-confidence predictions, the bottom left picture is missed detections, and the bottom right picture is high-confidence predictions.

In [ ]:
def create_diagnostic_mosaic(images_folder, weights_path, save_dir='diagnostics_output', output_name='diagnostic_mosaic.jpeg'):
    os.makedirs(save_dir, exist_ok=True)

    model = YOLO(weights_path)
    label_map = {0: "Blobs", 1: "Cracks", 2: "Spaghetti", 3: "Stringging", 4: "Under Extrusion"}

    misclassified_imgs = []
    low_conf_imgs = []
    no_detection_imgs = []
    high_conf_imgs = []

    for image_name in os.listdir(images_folder):
        if not image_name.endswith(('.jpg', '.png')):
            continue

        image_path = os.path.join(images_folder, image_name)
        image = cv2.imread(image_path)
        results = model(image, conf=0.15)
        pred = results[0]

        pred_boxes = pred.boxes.xywhn.cpu().numpy() if pred.boxes else np.array([])
        pred_classes = pred.boxes.cls.cpu().numpy().astype(int) if pred.boxes else np.array([])
        pred_conf = pred.boxes.conf.cpu().numpy() if pred.boxes else np.array([])

        gt_path = image_path.replace('images', 'labels').replace('.jpg', '.txt')
        if not os.path.exists(gt_path):
            continue

        ground_truth_boxes = []
        ground_truth_classes = []
        with open(gt_path, 'r') as f:
            for line in f:
                cls_id, x, y, w, h = map(float, line.strip().split())
                ground_truth_boxes.append([x, y, w, h])
                ground_truth_classes.append(int(cls_id))

        # no predictions but ground truth exists
        if pred_boxes.shape[0] == 0 and len(ground_truth_boxes) > 0:
            annotated_img, _ = draw_boxes_with_iou(image_path, pred_boxes, pred_classes,
                                                   ground_truth_boxes, ground_truth_classes,
                                                   label_map, confidences=[])
            no_detection_imgs.append(annotated_img)
            continue

        if pred_boxes.shape[0] > 0 and len(ground_truth_boxes) > 0:
            annotated_img, cls_matches = draw_boxes_with_iou(image_path, pred_boxes, pred_classes,
                                                             ground_truth_boxes, ground_truth_classes,
                                                             label_map, confidences=pred_conf)

            for match_id, pred_cls, gt_cls, _ in cls_matches:
                if pred_cls != gt_cls:
                    misclassified_imgs.append(annotated_img)
                    break
                elif pred_cls == gt_cls and pred_conf[match_id] < 0.3:
                    low_conf_imgs.append(annotated_img)
                    break
                elif pred_cls == gt_cls and pred_conf[match_id] >= 0.7:
                    high_conf_imgs.append(annotated_img)


        # early break if we have at least one for each category
        if (len(misclassified_imgs) > 0 and len(low_conf_imgs) > 0 and
            len(no_detection_imgs) > 0 and len(high_conf_imgs) > 0):
            break

    # pick random images or fallback to white canvas
    def pick_random_or_blank(lst):
        if len(lst) > 0:
            return random.choice(lst)
        else:
            return np.ones((384, 512, 3), dtype=np.uint8) * 255

    mosaic_images = [
        pick_random_or_blank(misclassified_imgs),
        pick_random_or_blank(low_conf_imgs),
        pick_random_or_blank(no_detection_imgs),
        pick_random_or_blank(high_conf_imgs)
    ]

    resized_images = [cv2.resize(img, (512, 384)) for img in mosaic_images]
    top_row = np.hstack(resized_images[:2])
    bottom_row = np.hstack(resized_images[2:])
    mosaic = np.vstack([top_row, bottom_row])

    output_path = os.path.join(save_dir, output_name)
    cv2.imwrite(output_path, mosaic)
    print(f"Mosaic saved to: {output_path}")


In [ ]:
create_diagnostic_mosaic(
    images_folder='/content/3d-fdm-failures-7/valid/images',
    weights_path='/content/drive/MyDrive/BP_Experiments/Adamax_DropOut_02_yolo_s/weights/best.pt',
    output_name='Adamax_DropOut_03_diagnostic_mosaic.jpeg'
)

In [ ]:
# runs create_diagnostic_mosaic for all models inside BP_Experiments
def run_all_diagnostics(base_experiments_dir, images_folder, save_dir='diagnostics_output'):
    os.makedirs(save_dir, exist_ok=True)

    for experiment in os.listdir(base_experiments_dir):
        experiment_path = os.path.join(base_experiments_dir, experiment)
        model_path = os.path.join(experiment_path, 'weights', 'best.pt')

        if not os.path.isfile(model_path):
            print(f"Skipping {experiment} – no best.pt found.")
            continue

        print(f"Running diagnostics for: {experiment}")
        create_diagnostic_mosaic(
            images_folder=images_folder,
            weights_path=model_path,
            save_dir=save_dir,
            output_name=f"{experiment}_diagnostic_mosaic.jpeg"
        )

run_all_diagnostics(
  base_experiments_dir='/content/drive/MyDrive/BP_Experiments',
  images_folder='/content/3d-fdm-failures-7/valid/images'
)


# **COMPARISON SCRIPT**

Simple script, which takes 2 versions of models, throws plot of their predictions on same image

In [ ]:
def load_ground_truth_boxes(image_path):
    ground_truth_path = image_path.replace('.jpg', '.txt').replace('images', 'labels')
    ground_truth_boxes = []
    ground_truth_classes = []

    with open(ground_truth_path, 'r') as f:
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.split())
            ground_truth_boxes.append([x_center, y_center, width, height])
            ground_truth_classes.append(int(class_id))

    return np.array(ground_truth_boxes), ground_truth_classes

def compare_models_on_image(model_path1, model_path2, image_path, conf_threshold=0.0):

    model1 = YOLO(model_path1)
    model2 = YOLO(model_path2)

    image = cv2.imread(image_path)
    ground_truth_boxes, ground_truth_classes = load_ground_truth_boxes(image_path)

    label_map = {0: "Blobs", 1: "Cracks", 2: "Spaghetti", 3: "Stringging", 4: "Under Extrusion"}

    results1 = model1(image, conf=conf_threshold)
    results2 = model2(image, conf=conf_threshold)

    predicted_boxes1 = results1[0].boxes.xywhn.cpu().numpy()
    predicted_classes1 = results1[0].boxes.cls.cpu().numpy()
    predicted_confidence1 = results1[0].boxes.conf.cpu().numpy()

    predicted_boxes2 = results2[0].boxes.xywhn.cpu().numpy()
    predicted_classes2 = results2[0].boxes.cls.cpu().numpy()
    predicted_confidence2 = results2[0].boxes.conf.cpu().numpy()

    image_with_boxes1 = draw_boxes_with_iou(
        image, predicted_boxes1, predicted_classes1, ground_truth_boxes, ground_truth_classes, label_map, confidences=predicted_confidence1
    )
    image_with_boxes2 = draw_boxes_with_iou(
        image, predicted_boxes2, predicted_classes2, ground_truth_boxes, ground_truth_classes, label_map, confidences=predicted_confidence2
    )

    fig, axs = plt.subplots(1, 2, figsize=(15, 7))
    axs[0].imshow(cv2.cvtColor(image_with_boxes1, cv2.COLOR_BGR2RGB))
    axs[0].set_title("Model 1 Predictions")
    axs[0].axis('off')

    axs[1].imshow(cv2.cvtColor(image_with_boxes2, cv2.COLOR_BGR2RGB))
    axs[1].set_title("Model 2 Predictions")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
model1_path = '/content/drive/MyDrive/Default_Train_pr/run_2/weights/best.pt'
model2_path = '/content/drive/MyDrive/NoAug_NoMosaic_CosLR_pr/300_epochs_run/weights/best.pt'
image_path = '/content/3d-fdm-failures-6/valid/images/5-20-20-_jpg.rf.4b297853959ddedf6858816dc358ed22.jpg'
compare_models_on_image(model1_path, model2_path, image_path)

# **EVALUATION TABLE SCRIPT**

Similar script to train script, but runs evaluation on each configuration

In [ ]:
def load_yaml_config(yaml_path):
    with open(yaml_path, 'r') as f:
        return yaml.safe_load(f)


def get_model_checkpoint(experiment_root):
    model_path = os.path.join(experiment_root, "weights", "best.pt")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model best.pt not found: {model_path}")
    return model_path


def evaluate_yolo_model(model_path, data_path, save_dir=None):
    model = YOLO(model_path)
    results = model.val(data=data_path, save=True, save_dir=save_dir)
    print(f"Evaluation completed for {model_path} completed")
    return results


def run_all_evaluations(experiment_root_dir, config_dir, dataset_root=None):
    for file in os.listdir(config_dir):
        if file.endswith('.yaml'):
            config_path = os.path.join(config_dir, file)
            config = load_yaml_config(config_path)

            name = config.get('name')
            data_path = f"{dataset.location}/data.yaml"

            experiment_dir = os.path.join(experiment_root_dir, name)
            try:
                model_path = get_model_checkpoint(experiment_dir)
                print(f"Evaluating {model_path} ...")
                evaluate_yolo_model(model_path, data_path, save_dir=os.path.join(experiment_dir, 'val_results'))
            except Exception as e:
                print(f"Evaluation failed for {model_path}: {e}")

# **ANNOTATIONS CONVERSION**

Script which converts existing segmentation annotations to bounding boxes

In [ ]:
import os
from PIL import Image

def convert_segmentation_to_yolo(input_folder, output_folder, image_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for label_file in os.listdir(input_folder):
        if label_file.endswith(".txt"):

            image_filename = os.path.splitext(label_file)[0] + '.jpg'
            image_path = os.path.join(image_folder, image_filename)

            with Image.open(image_path) as img:
                image_width, image_height = img.size

            with open(os.path.join(input_folder, label_file), 'r') as f:
                lines = f.readlines()

            #print(f"IMAGE WORKING ON: {image_filename} WIDTH: {image_width} HEIGHT: {image_height}")

            new_labels = []
            for line in lines:
                data = line.strip().split()
                class_id = int(data[0])
                coords = list(map(float, data[1:]))

                x_coords = coords[0::2]
                y_coords = coords[1::2]

                x_min = min(x_coords)
                x_max = max(x_coords)
                y_min = min(y_coords)
                y_max = max(y_coords)

                #print(f"X_MIN: {x_min} X_MAX: {x_max}\nY_MIN: {y_min} Y_MAX: {y_max}")

                center_x = (x_min + ((x_max - x_min)/2))
                center_y = (y_min + ((y_max - y_min)/2))
                box_width = (x_max - x_min)
                box_height = (y_max - y_min)

                new_label = f"{class_id} {center_x:.6f} {center_y:.6f} {box_width:.6f} {box_height:.6f}"
                #print(new_label)
                new_labels.append(new_label)

            output_file = os.path.join(output_folder, label_file)
            with open(output_file, 'w') as f_out:
                f_out.write("\n".join(new_labels))

    print(f"Converted labels saved to {output_folder}")

input_folder = '/content/DATASET_CUSTOMIZADO_TCC/train/labels'  # Folder with segmentation labels
output_folder = '/content/Dataset_Converted/train/labels'  # Folder to save YOLO labels
image_folder = '/content/DATASET_CUSTOMIZADO_TCC/train/images'  # Folder where corresponding images are stored

convert_segmentation_to_yolo(input_folder, output_folder, image_folder)


# **ALBLUMENTATIONS CHECK**

Script created purely to understand how on-fly augmentations affect image

In [ ]:
import albumentations as A

#Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

image = cv2.imread('/content/3d-fdm-failures-6/train/images/a_jpg.rf.d71fc3017c62a4cc48cf3e0508ea82ee.jpg')
image_height = image.shape[0]
image_width = image.shape[1]

albu_pipeline = A.Compose([
    # A.Crop(x_min=0, y_min=0, x_max=image_width // 2, y_max=image_height // 2)
    A.Blur(p=1, blur_limit=(3, 7)),
    A.MedianBlur(p=1, blur_limit=(3, 7)),
    A.ToGray(p=1, num_output_channels=3, method='weighted_average'),
    A.CLAHE(p=1, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
])

augmented_image = albu_pipeline(image=image)["image"]



fig, axs = plt.subplots(1, 2, figsize=(12, 6))


axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title("Original")

axs[1].imshow( cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB))
axs[1].set_title("Augmented")

for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()

# **DUPLICATES DETECTION**

Script which compares naming of images to detect possible duplicates of images

In [ ]:
# extract the part of the filename before %.rf.%
def extract_base_name(filename):
    return filename.split('.rf.')[0]

# calculate the percentage similarity between two strings
def calculate_similarity(name1, name2):
    return SequenceMatcher(None, name1, name2).ratio() * 100

def find_similar_files(valid_folder, test_folder, threshold=80, save_to_excel=False, output_file="similar_files.xlsx"):
    valid_files = [f for f in os.listdir(valid_folder) if os.path.isfile(os.path.join(valid_folder, f))]
    test_files = [f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder, f))]

    similar_files = []  # list to store results

    # compare files based on base names
    for valid_file in valid_files:
        valid_base = extract_base_name(valid_file)
        for test_file in test_files:
            test_base = extract_base_name(test_file)
            similarity = calculate_similarity(valid_base, test_base)
            if similarity >= threshold:
                similar_files.append([valid_file, test_file, round(similarity, 2)])

    df = pd.DataFrame(similar_files, columns=["Validation File", "Test File", "Similarity (%)"])

    if not df.empty:
        print(f"\nFiles with similarity >= {threshold}% based on base names:\n")
        print(df.to_string(index=False))
        if save_to_excel:
            df.to_excel(output_file, index=False)
            print(f"\nResults saved to {output_file}")
    else:
        print(f"No files found with similarity >= {threshold}% based on base names.")

valid_folder = "/content/3d-fdm-failures-6/valid/images"
test_folder = "/content/3d-fdm-failures-6/train/images"
similarity_threshold = 100  # threshold for similarity
find_similar_files(valid_folder, test_folder, similarity_threshold, save_to_excel=True)


# **EXPERIMENTS**

This section contains script for executing experiments. Experiments are configured in .yaml files on Google Drive.

Script iterates through all .yaml files, trains appropriate model and moves results to Google Drive for further analysis.

**Note:** make sure to run cell with `!pip install`, since numpy ocasionally causes errors

In [ ]:
!pip install --upgrade --force-reinstall numpy --no-cache-dir
!pip install --upgrade --force-reinstall --no-cache-dir ultralytics

In [ ]:
# load .yaml config file
def load_yaml_config(yaml_path):
    with open(yaml_path, 'r') as f:
        return yaml.safe_load(f)

# select yolov8 size for experiment
def select_model_from_filename(filename):
    return 'yolov8s.pt' if 'YOLO_S' in filename else 'yolov8n.pt'

# initiate training with selected cfg
def train_yolo_model(config_path, config):
    model_path = select_model_from_filename(os.path.basename(config_path))
    model = YOLO(model_path)

    name = config.get('name', 'default_run')
    project = config.get('project', 'default_project')
    data_path = f"{dataset.location}/data.yaml"

    print(f"Starting Experiment: {name}")
    model.train(
        data=data_path,
        cfg=config_path,
        project=project,
        name=name,
    )
    return project, name

# move result of experiment to google drive
def move_run_output(project, name, dest_root):
    src = os.path.join('/content', project, name)
    dst = os.path.join(dest_root, name)
    if os.path.exists(src):
        shutil.move(src, dst)
        print(f"Moved: {src} to {dst}")
    else:
        print(f"Output folder not found: {src}")

# main loop for experiments training
def run_all_experiments(config_dir, final_output_dir):
    for file in os.listdir(config_dir):
        if file.endswith('.yaml'):
            config_path = os.path.join(config_dir, file)
            try:
                config = load_yaml_config(config_path)
                project, name = train_yolo_model(config_path, config)
                move_run_output(project, name, final_output_dir)
            except Exception as e:
                print(f"Exception at {file}: {e}")


CONFIG_DIR = '/content/drive/MyDrive/Experiments_YAML_files'
OUTPUT_DIR = '/content/drive/MyDrive/BP_Test'
run_all_experiments(CONFIG_DIR, OUTPUT_DIR)


# **THESIS IMAGES MANIPULATIONS**

Section with helped with merging images

In [ ]:
def resize_keep_ratio(img, height):
    scale = height / img.shape[0]
    width = int(img.shape[1] * scale)
    return cv2.resize(img, (width, height))

def combine_graphs(img1_path, img2_path, output_path='combined.jpg', remove_legend_from='right', legend_coords=None, remove_pixels=400):

    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    if img1 is None or img2 is None:
        raise FileNotFoundError("One or both image paths are incorrect.")

    # remove legend
    if remove_legend_from and legend_coords:
        x1, y1, x2, y2 = legend_coords
        if remove_legend_from.lower() == 'left':
            img1[y1:y2, x1:x2] = 255
        elif remove_legend_from.lower() == 'right':
            img2[y1:y2, x1:x2] = 255

    # resize to the same height
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    max_height = max(h1, h2)

    img1_resized = resize_keep_ratio(img1, max_height)
    img2_resized = resize_keep_ratio(img2, max_height)

    # cut out extra space from right side of left image
    img1_resized = img1_resized[:, :-remove_pixels]

    combined = np.hstack((img1_resized, img2_resized))

    # Save as JPEG
    final_image = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(final_image)
    pil_img.save(output_path, format='JPEG')

    print(f"image saved to {output_path}")

combine_graphs('/content/F1_Adamax_YOLO_S.jpg', '/content/PR_Adamax_YOLO_S.jpg', 'F1_Adamax_YOLO_S_PR_Adamax_YOLO_S.jpg', remove_legend_from='left', legend_coords=(1857, 115,2485, 795))


In [ ]:
def batch_combine_graphs(f1_dir, pr_dir, output_dir, legend_coords=None, remove_legend_from='right', remove_pixels=50):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    f1_files = [f for f in os.listdir(f1_dir) if f.startswith('F1_') and f.endswith('.jpg')]

    for f1_file in f1_files:
        base_name = f1_file.replace('F1_', '')
        pr_file = f'PR_{base_name}'

        f1_path = os.path.join(f1_dir, f1_file)
        pr_path = os.path.join(pr_dir, pr_file)

        if os.path.exists(pr_path):
            output_filename = f'combined_{f1_file.replace(".jpg", "")}_{pr_file}'
            output_path = os.path.join(output_dir, output_filename)

            try:
                combine_graphs(f1_path, pr_path, output_path=output_path,
                               remove_legend_from=remove_legend_from,
                               legend_coords=legend_coords,
                               remove_pixels=remove_pixels)
            except Exception as e:
                print(f"Failed to combine {f1_file} and {pr_file}: {e}")
        else:
            print(f"PR image not found for: {f1_file}")


batch_combine_graphs('/content/Experiment_Results_JPG/F1', '/content/Experiment_Results_JPG/PR', 'fixed', legend_coords=(1564, 80, 1875, 938), remove_legend_from='left', remove_pixels=300)


In [ ]:
def resize_and_embed_legend(image_path, legend_coords, scale_factor=1.5):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return

    x1, y1, x2, y2 = legend_coords
    legend = image[y1:y2, x1:x2]
    legend_resized = cv2.resize(legend, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

    h_new, w_new = legend_resized.shape[:2]
    h_img, w_img = image.shape[:2]

    new_canvas = np.ones((max(h_img, y1 + h_new), max(w_img, x1 + w_new), 3), dtype=np.uint8) * 255
    new_canvas[:h_img, :w_img] = image
    new_canvas[y1:y1 + h_new, x1:x1 + w_new] = legend_resized

    output_path = image_path.replace('.jpg', '.jpg')
    cv2.imwrite(output_path, new_canvas)
    return output_path


def batch_combine_graphs(f1_dir, pr_dir, output_dir, legend_coords=None, remove_legend_from='right', zoom_legend=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    f1_files = [f for f in os.listdir(f1_dir) if f.startswith('F1_') and f.endswith('.jpg')]

    for f1_file in f1_files:
        base_name = f1_file.replace('F1_', '')
        pr_file = f'PR_{base_name}'

        f1_path = os.path.join(f1_dir, f1_file)
        pr_path = os.path.join(pr_dir, pr_file)

        if os.path.exists(pr_path):
            output_filename = f'{f1_file.replace(".jpg", "")}_{pr_file}'
            output_path = os.path.join(output_dir, output_filename)

            try:
                combine_graphs(f1_path, pr_path, output_path=output_path,
                               remove_legend_from=remove_legend_from,
                               legend_coords=legend_coords)

                if zoom_legend and legend_coords:
                    resize_and_embed_legend(output_path, legend_coords)

            except Exception as e:
                print(f"Failed to combine {f1_file} and {pr_file}: {e}")
        else:
            print(f"PR image not found for: {f1_file}")


batch_combine_graphs(
    f1_dir='/content/Experiment_Results_JPG/F1',
    pr_dir='/content/Experiment_Results_JPG/PR',
    output_dir='/content/combined',
    legend_coords=(4125, 100, 4687, 830),
    remove_legend_from='left',
    zoom_legend=True
)


In [ ]:
resize_and_embed_legend('/content/F1_Baseline_YOLO_S_PR_Baseline_YOLO_S.jpg', (3965, 105, 4600, 775))